In [1]:
%%capture
import stata_setup, os
if os.name == 'nt':
    stata_setup.config('C:/Program Files/Stata17/','mp')
else:
    stata_setup.config('/usr/local/stata17','mp')

## Preparing the data

In [2]:
%%stata -qui

use "../data/data", clear
rename log_flesch_kincaid_grade_level FKG
quietly tabulate year, generate(y_)
quietly tabulate cluster, generate(c_)

local journals  ecm jpe qje res  //AER based category

local jel_imp a_imp b_imp c_imp  e_imp f_imp g_imp h_imp i_imp j_imp k_imp /// 
		l_imp m_imp n_imp o_imp p_imp q_imp r_imp y_imp z_imp // D JEL based case

## Post-Regularization Lasso (lo correcto es double debiased machine learning)

The Double/Debiased Machine Learning estimator is implemented by the ```xporegress``` Stata command. It'll utilize a 'plugin' value for the $\lambda$s parameters by default, but they can also use standard cross-validation instead.

In [3]:
%%stata -qui
#delimit ;
xporegress FKG  log_num_authors log_num_pages both_genders prop_women,
			controls(`journals' `jel_imp' y_2-y_20  c_2-c_215  jel_flag)
            vce(cluster cluster) rseed(42);
estimates store xpo_plugin;
xporegress FKG  log_num_authors log_num_pages both_genders prop_women,
			controls(`journals' `jel_imp' y_2-y_20  c_2-c_215  jel_flag)
            vce(cluster cluster) selection(cv) rseed(42);
estimates store xpo_cv;
#delimit cr
# la lista de variables, ciertos coeficientes no son diferente de cero (esparcidad, sparcity)
# permite estimar los coeficientes de una forma consistente y hacer inferencia. Diferente a un modelo de lasso para predecir.

In [4]:
%%stata
estimates restore xpo_cv
xporegress
# la elasticidad entre la proporcion de mujeres y el abstracto de entendimiento del texto es de 1.64
# solo podemos hacer inferencia para regresores que incluidos en el modelo como regresores D (41 obs)
# si lo estimamos no podemos hacer inferencia de ello, los modelos de regresion lineal.
#Los modelos no lineales, logistica no existe efectos fijos de alta dimensionalidad, para poisson si existe efectos fijos de alta dimensionaldiad
# podemos ampliar para modelos no lineales? la respuesta es si.


. estimates restore xpo_cv
(results xpo_cv are active now)

. xporegress

Cross-fit partialing-out             Number of obs                =      4,988
linear model                         Number of controls           =        257
                                     Number of selected controls  =         41
                                     Number of folds in cross-fit =         10
                                     Number of resamples          =          1
                                     Wald chi2(4)                 =       7.80
                                     Prob > chi2                  =     0.0993

                              (Std. err. adjusted for 215 clusters in cluster)
------------------------------------------------------------------------------
             |               Robust
         FKG | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
log_num_au~s |  -.0